In [3]:
import ee
import os
import math
from datetime import datetime
from src.utils import ee_treatments
from src.utils import runs
from src.utils.check_exists import check_exists
from src.utils.yml_params import get_export_params
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize(project='mas-gee')
%load_ext autoreload
%autoreload 2

# In development phase
tag = 'ee_treatments_production'
ee.data.setWorkloadTag(tag)

## Million Acres Strategy - Treatment Simulator
#### Creates Randomized Treatment Scenario Landscapes - exports as ee.Images
#### This Notebook is for Run IDs: 
|RunID | CA_REGION | INTENSITY_ID | PRIORITY |
|------|-----------|--------------|----------|
| 27  | South Coast | 2m   | Fire |
| 36 | Central Coast   | 2m   | Fire |

Run Settings: Change `RUNID` below to setup the analysis and output file names

In [4]:
RUNID = 'RunID27' 

Auto-calculate the necessary information for the Scenario

Double-Check the Output Calcs are Correct against the [Spreadsheet](https://docs.google.com/spreadsheets/d/1Gnl6SO5kOkj4Ne1JdXzW4bp03824Zb1Cp_2HVuCY3Lk/edit#gid=0
)

In [5]:
# string identifiers for output file
CA_REGION = runs.key[RUNID]['RegionID'] # NC = North Coast; SN = Sierra Nevada
INTENSITY_ID = runs.key[RUNID]['IntensityID'] 
PRIORITY = runs.key[RUNID]['PriorityID'] 

# construct output path
today_string = datetime.utcnow().strftime("%Y-%m-%d").replace("-", "")
output_image = f"projects/mas-gee/assets/treatment_scenarios/{RUNID}/{CA_REGION}_{PRIORITY}_{INTENSITY_ID}_{today_string}"
print(f'Output EE Image Path: {output_image}')

# treatment size is static and set in runs.py
treatment_size_ac = runs.treatment_size_ac 
TREATMENT_SIZE = runs.treatment_size_sqm 
print('TREATMENT SIZE (m^2):',TREATMENT_SIZE)
print('TREATMENT SIZE (Ac):',treatment_size_ac)
RADIUS = math.sqrt(TREATMENT_SIZE)/2  # in meters, rough square radius is A = (side/2) solve for side (side = sqrt(A) )

sqm_to_ac_x = 0.000247105

# pull prescribed treatment area in both units, we use sqm for analysis and acreage for reporting
total_treated_area = runs.key[RUNID]['sqmPerYear']
print(f"Area to Treat Per Year (m^2): {total_treated_area}")
total_treated_area_ac = runs.key[RUNID]['acreagePerYear']
print(f"Area to Treat Per Year (Ac): {total_treated_area_ac}")

# we run the treatment generator as 5-year interval outputs (4 run intervals covering the 20-year period)
# so we actually need to total_treated_area*5 to the ee_treatments() generator
area_per_iteration = int(round(total_treated_area*5))
print(f'Area to treat per 5-year interval (m^2): {area_per_iteration}')
area_per_iteration_ac = int(round(area_per_iteration*sqm_to_ac_x))
print(f'Area to treat per 5-year interval (Ac): {area_per_iteration_ac}')

Output EE Image Path: projects/mas-gee/assets/treatment_scenarios/RunID27/SC_Fire_2m_20231020
TREATMENT SIZE (m^2): 404685.64224
TREATMENT SIZE (Ac): 100
Area to Treat Per Year (m^2): 1489243163.4432
Area to Treat Per Year (Ac): 368000
Area to treat per 5-year interval (m^2): 7446215817
Area to treat per 5-year interval (Ac): 1839997


Input Files

In [6]:
# Sierra Nevada(SN), South Coast(SC), Central Coast(CC), North Coast(NC)
all_hucs = ee.FeatureCollection("projects/mas-gee/assets/TxHucScCc")
# property names differ between Wui, BP, and RFFc

# Treatable Veg Constraint Layer
# is result of Softwoods OR Road Grids 
# dev note - keep in mind there's some false positive data in eastern portion of extent due to Softwood layer being all of CA
# but ee_treatments should handle that fine using the extent of the HUC groups which excludes those areas
veg_constraint = ee.Image("projects/mas-gee/assets/softwoodsOrRoadGridsImg")


Specific Set up for North Coast and Sierra Nevada WUI Prioritization. 

see [Treatment Allocation Runs sheet](https://docs.google.com/spreadsheets/d/1Gnl6SO5kOkj4Ne1JdXzW4bp03824Zb1Cp_2HVuCY3Lk/edit#gid=0)

* 1-25th percentile HUC group in Year Interval 1 and 4
* 25th-50th percentile HUC group in Year Interval 2
* 50-75th percentile HUC group in Year Interval 3.

In [7]:
year_interval_ids = ee.List(['Y1to5',
                             'Y6to10',
                             'Y11to15',
                             'Y16-20'
                             ]
                             )
huc_filters = [ee.Filter.eq('TxBpPrc', 25),
               ee.Filter.eq('TxBpPrc', 50),
               ee.Filter.eq('TxBpPrc', 75),
               ee.Filter.eq('TxBpPrc', 100)
               ]
# zipping ee.Lists is a good way to parallelize complicated things in GEE with .map()
zipped = year_interval_ids.zip(huc_filters)

Pay attention to `overshoot` value passed in `ee_treatments()` call. It is your dial. 

In [8]:
PT_OVERSHOOT = 2.8

In [9]:
# for QA test runs, get biggest and largest HUC poly per percentile group, may need to coded into this treat_by_year_huc() 
def treat_by_year_huc(l):
    """
    this is to be map()ed across zipped (year intervals and percentile huc filters) to parallelize use of ee_treatments()
        result is one ee.Image of treatment areas per yearInterval/HUC Group element in zipped     
    """
    zipped_list = ee.List(l)
    ranks_filter = ee.Filter(zipped_list.get(1))
    
    # Get HUC group
    huc_group = (all_hucs.filter(ee.Filter.eq('RRK_Rgn', runs.key[RUNID]['Region Filter Name']))
                .filter(ranks_filter)
                # .limit(2) # for testing, reduce compute for development
                ) 
    
    # area_per_iteration is the prescribed area 
    # area_per_iteration_testing = area_per_iteration*(2/193) 
    
    pixel_value_iteration = ee.Number(ee.List(zipped).indexOf(l)).add(1)
    
    # this is for reshuffling the random treatment placements, may be useful for re-treatments of the same huc groups
    # we would pass a unique number to 'seed' arg in ee_treatments() each time 
    # if you don't need to reshuffle the random seed each run, don't define 'seed' arg and function will use its default seed every time
    seeds = ee.List([10,20,30,40])
    seed = ee.Number(ee.List(seeds).get(ee.List(zipped).indexOf(l)))
    
    treated_area_img, properties = ee_treatments.ee_treatments(hucs=huc_group,
                                                  prescription=area_per_iteration, # area_per_iteration_testing , reduce compute for testing
                                                  unit_size=TREATMENT_SIZE,
                                                  radius=RADIUS,
                                                  pixel_value=pixel_value_iteration,
                                                  constraint_layer=veg_constraint,
                                                  # seed=seed # passing seed defined as diff number for every ee_treatment() run to randomize re-treatment groups
                                                  ptOvershoot=PT_OVERSHOOT # we dial this in per scenario to get close to a 1:1 of Needed:Generated
                                                  )
    return treated_area_img,properties

def remap_keys(d):
  """
  solves a QA property problem for ee_treatments(): we have duplicate key names in the properties dict 
  when multiple ee_treatments() results are combined together.
  So we create a unique identifier from 'yearInterval' and remove 'yearInterval' as a property itself.
  example: 'PropertyName': PropertyValue becomes 'yearInterval1PropertyName': PropertyValue
  """
  d = ee.Dictionary(d)
  yearInterval = (ee.String('yearInterval')
  .cat(ee.Number(ee.Dictionary(d).get('yearInterval')).format()))
  keys = d.keys().slice(0,7) # depends on yearInterval property being in index position 7 of property list returned by ee_treatments()
  values = d.values().slice(0,7)
  remapped_keys = keys.map(lambda s: ee.String(yearInterval).cat(s))
  return ee.Dictionary.fromLists(remapped_keys,values)

def combine_dicts(d):
   d = ee.Dictionary(d)
   return d.keys().zip(d.values())

Create treated area ee.Images for every year interval on its assigned group of HUCs

In [10]:
# We use .map() to parallelize computation of treatment area images across the four 5-year intervals, 
# treating equal amount of area within each percentile ranking list, returning 4 treatment ee.Images, 
# then mosaicking them into one ee.Image for export

# returns ee.List((ee.Image1,ee.Dictionary1),(ee.Image2,ee.Dictionary2),(etc))
results = ee.List(zipped).map(treat_by_year_huc) 

# parse the images and properties out of the (image,properties) tuples by .map()ing over results
images = results.map(lambda t: ee.List(t).get(0)) 
properties = results.map(lambda t: ee.List(t).get(1))

# remap the keys of the properties so they are unique per ee_treatments() image run, then combine into one properties dict
all_properties = properties.map(remap_keys).map(combine_dicts).flatten()
# print(all_properties.getInfo())

# mosaic all ee_treatment() images together and set the fixed properties 
output = ee.Image(ee.ImageCollection.fromImages(images).mosaic()).set(all_properties)



Can export just the ee_treatment image to check it out, not the final product though

In [11]:

# export
desc = f'{RUNID}_{os.path.basename(output_image)}'

folder = os.path.dirname(output_image)
if check_exists(folder):
    print(f'making new folder: {folder}')
    os.popen(f'earthengine create folder {folder}').read()
ee_treatments.export_img(output,desc,output_image,all_hucs.geometry(),30,'EPSG:5070')

Export Started for projects/mas-gee/assets/treatment_scenarios/RunID27/SC_Fire_2m_20231020


### Now the retreatment code

To get close to hitting 2.3M target we need to introduce a re-treatment of the Road Grid areas for percentile groups 1,2, and 4 (matching the GridFire Runs of Year Interval 1 (years 1-5), YI 10 (years 6-10) and YI20 (years 15-20))

For those percentile HUC groups we clip the Road Grid image to each HUC group boundary, then set pixel value to the representative percentile year group (1,2, or 4)

Then we mosaic this 're-treat' image with the orignal ee_treatments() image and export the result

In [24]:
# get road grids image for SC or CC
road_grids = ee.Image(runs.key[RUNID]['RoadGrids'] )

# YI 1, 2, and 4 not all 4 like before
# For WUI, we know that YI 1 and YI 4 HUC groups share same footprint, 
# so all pixels in these two 'treatment' YIs have pixel value 4, its a limitation and FCAT deals with that.
year_interval_ids = ee.List(['Y1to5',
                             'Y6to10',
                             'Y16-20'])
ranked_huc_filters = [ee.Filter.eq('TxBpPrc', 25),
               ee.Filter.eq('TxBpPrc', 50),
               ee.Filter.eq('TxBpPrc', 100)
               ]
# zipping ee.Lists is a good way to parallelize complicated things in GEE with .map()
zipped = year_interval_ids.zip(ranked_huc_filters)

# define similar per HUC group function as used for ee_treatments mapping
def treat_by_year_huc_road_grid(l):
    """
    this is to be map()ed across zipped (year intervals and percentile huc filters) to parallelize use of ee_treatments()
        result is one ee.Image of treatment areas per yearInterval/HUC Group element in zipped     
    """
    zipped_list = ee.List(l)
    ranks_filter = ee.Filter(zipped_list.get(1))
    
    # Get HUC group
    huc_group = (all_hucs.filter(ee.Filter.eq('RRK_Rgn', runs.key[RUNID]['Region Filter Name']))
                .filter(ranks_filter)
                ) 
    # rather than passing in another list zipped up in mappable object we just retrieve the right pixel value 
    # (1,2,4) for each image in the list by indexing the original zipped list, 
    # then retrieving pixel value at same index position
    index_pixel_values = ee.List([1,2,4])
    index_l = zipped.indexOf(l)
    pixel_value = index_pixel_values.get(index_l)
    road_grid_huc = (road_grids.unmask(0) # ensure its binary 1/0
                     .clip(huc_group)
                     .multiply(ee.Number(pixel_value)))
    return road_grid_huc.set('index_l',index_l,'pixel_value',pixel_value)
    
# .map() the by-year by-huc group function 
road_grid_treated = ee.List(zipped).map(treat_by_year_huc_road_grid) 
print(ee.Image(ee.List(road_grid_treated).get(2)).get('index_l').getInfo())
print(ee.Image(ee.List(road_grid_treated).get(2)).get('pixel_value').getInfo())

# pull out each road_grid image with its designated pixel value representing YI 1, YI 2, YI 4
road_grid_1 = ee.Image(road_grid_treated.get(0))
road_grid_2 = ee.Image(road_grid_treated.get(1))
road_grid_4 = ee.Image(road_grid_treated.get(2))

# composite them using max reducer
road_grid_combined = ee.Image((road_grid_1
                      .addBands(road_grid_2)
                      .addBands(road_grid_4))
                      .reduce(ee.Reducer.max())
                      .rename('Tx')
                      .selfMask())



Can export just the road grid retreated img to check it out. Not the final product though.

In [25]:
# export
# desc = f'{RUNID}_{os.path.basename(output_image)}_retreatment'

# folder = os.path.dirname(output_image)
# if check_exists(folder):
#     print(f'making new folder: {folder}')
#     os.popen(f'earthengine create folder {folder}').read()
# ee_treatments.export_img(road_grid_combined,desc,output_image+'retreatment',all_hucs.geometry(),30,'EPSG:5070')

In [26]:
# import geemap
# Map = geemap.Map()

# Map.addLayer(road_grids,{},'Road Grid Img')
# Map.addLayer(ee.Image(road_grid_treated.get(0)),{'min':1,'max':4,'palette':['blue','green','red']},'road_grid_treated 1 img')
# Map.addLayer(ee.Image(road_grid_treated.get(1)),{'min':1,'max':4,'palette':['blue','green','red']},'road_grid_treated 2 img')
# Map.addLayer(ee.Image(road_grid_treated.get(2)),{'min':1,'max':4,'palette':['blue','green','red']},'road_grid_treated 3 img')
# Map.addLayer(road_grid_combined,{'min':1,'max':4,'palette':['blue','green','red']},'output road grid reduced')

# scenario_hucs = all_hucs.filter(ee.Filter.eq('RRK_Rgn', runs.key[RUNID]['Region Filter Name']))
# scn_hucs_y1 = scenario_hucs.filter(ee.Filter.eq('TxWPrct', 25))
# scn_hucs_y2 = scenario_hucs.filter(ee.Filter.eq('TxWPrct', 50))
# scn_hucs_y4 = scenario_hucs.filter(ee.Filter.eq('TxWPrct', 25))
# Map.addLayer(scn_hucs_y1,{'color':'black'},'scenario HUCS 1')
# Map.addLayer(scn_hucs_y2,{'color':'black'},'scenario HUCS 2')
# Map.addLayer(scn_hucs_y4,{'color':'black'},'scenario HUCS 4')

# Map.centerObject(road_grids,10)
# Map

Final export is the ee_treatment() image mosaicked with the road_grid retreatment image 

In [27]:
# mosaic ee_treatments() and re-treatment road grid images
treat_retreat = output.addBands(road_grid_combined).reduce(ee.Reducer.max()).rename('Tx')

# export
desc = f'{RUNID}_{os.path.basename(output_image)}_ee_treatment_plus_retreatment'

folder = os.path.dirname(output_image)
if check_exists(folder):
    print(f'making new folder: {folder}')
    os.popen(f'earthengine create folder {folder}').read()
ee_treatments.export_img(treat_retreat,desc,output_image+'_ee_treatment_plus_retreatment',all_hucs.geometry(),30,'EPSG:5070')

making new folder: projects/mas-gee/assets/treatment_scenarios/RunID35
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID35/CC_RFFC_2m_20231017_ee_treatment_plus_retreatment
